In [ ]:
aoi_df = pd.read_csv("african_coordinates.csv")  # contient pays, lat, lon, année

ndvi_stats = []

for idx, row in aoi_df.iterrows():
    lat, lon = row["latitude"], row["longitude"]
    year = row["year"]
    lat_range = (lat - 0.05, lat + 0.05)
    lon_range = (lon - 0.05, lon + 0.05)
    time_range = (f"{year}-01-01", f"{year}-12-31")

    ds_ndvi = load_ndvi_africa(lat_range, lon_range, time_range)
    ndvi_mean = ds_ndvi.NDVI.mean(dim=["x", "y"]).mean().item()

    ndvi_stats.append({
        "country": row["country"],
        "year": year,
        "latitude": lat,
        "longitude": lon,
        "ndvi_mean": round(ndvi_mean, 4)
    })

ndvi_df = pd.DataFrame(ndvi_stats)


In [ ]:
df_agro = pd.read_csv("dataset_agricole_prepared.csv")
df_agro["latitude"] = df_agro["latitude"].round(2)
df_agro["longitude"] = df_agro["longitude"].round(2)

df_fusion = pd.merge(df_agro, ndvi_df, on=["country", "year", "latitude", "longitude"], how="inner")
print(f"✅ Fusion réussie : {len(df_fusion)} lignes")


In [ ]:
def load_ndvi_africa(lat_range, lon_range, time_range):
    ds = load_ard(
        dc=dc,
        products=["s2_l2a"],
        x=lon_range,
        y=lat_range,
        time=time_range,
        output_crs="EPSG:4326",
        cloud_mask=True,
        resolution=(-0.0001, 0.0001)
    )
    ds = calculate_indices(ds, index="NDVI", satellite_mission="s2")
    return ds


In [ ]:
from datacube import Datacube
from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

dc = Datacube(app="senesmart_africa_engine")
